***Departing time in time format***

In [ ]:

#Create table transformed_redbus with transformed_departing_time
CREATE TABLE trans_redbus AS
SELECT
    *,
    STR_TO_DATE(
        CASE
            WHEN departing_time LIKE '% %' THEN SUBSTRING_INDEX(departing_time, ' ', 1)
            ELSE departing_time
        END,
        '%H:%i'
    ) AS t_departing_time
FROM redbus_new_table
WHERE 
    CASE
        WHEN departing_time LIKE '% %' THEN SUBSTRING_INDEX(departing_time, ' ', 1)
        ELSE departing_time
    END REGEXP '^[0-2][0-9]:[0-5][0-9]$';

In [ ]:
##Create table b_routes with transformed reaching time
CREATE TABLE b_routes AS
SELECT 
    *,
    CASE
        WHEN 
            FLOOR(SUBSTRING_INDEX(SUBSTRING_INDEX(reaching_time, ' ', 1), 'h', 1)) < 24 AND
            FLOOR(SUBSTRING_INDEX(SUBSTRING_INDEX(reaching_time, ' ', -1), 'm', 1)) < 60
        THEN
            STR_TO_DATE(
                CONCAT(
                    LPAD(FLOOR(SUBSTRING_INDEX(SUBSTRING_INDEX(reaching_time, ' ', 1), 'h', 1)), 2, '0'), 
                    ':',
                    LPAD(FLOOR(SUBSTRING_INDEX(SUBSTRING_INDEX(reaching_time, ' ', -1), 'm', 1)), 2, '0')
                ),
                '%H:%i'
            )
        ELSE
            NULL  -- or some default value
    END AS t_reaching_time
FROM trans_redbus
WHERE reaching_time REGEXP '^[0-9]+h [0-9]+m$';

***Categorised bys_type***

In [ ]:
#Select everything from b_routes
#Categorize bus types in redbus_routes
SELECT 
    DISTINCT bus_type,
    CASE 
        WHEN LOWER(bus_type) LIKE '%non%' THEN "No"
        ELSE "Yes"
    END AS contains_ac,
    CASE 
        WHEN LOWER(bus_type) LIKE '%seater%' AND LOWER(bus_type) NOT LIKE '%sleeper%' THEN 'Seater'
        WHEN LOWER(bus_type) LIKE '%sleeper%' AND LOWER(bus_type) NOT LIKE '%seater%' THEN 'Sleeper'
        WHEN LOWER(bus_type) LIKE '%semi%' AND LOWER(bus_type) LIKE '%sleeper%' AND LOWER(bus_type) NOT LIKE '%seater%' THEN 'Semi Sleeper'
        WHEN LOWER(bus_type) LIKE '%semi%' AND LOWER(bus_type) LIKE '%seater%' AND LOWER(bus_type) NOT LIKE '%sleeper%' THEN 'Semi Seater'
        ELSE 'new_type'
    END AS bus_types_ss
FROM redbus_routes;

In [ ]:
##Create newbus_route table with standardized star ratings
CREATE TABLE newbus_route AS
SELECT
    *,
    CAST(standardized_star_ratings AS FLOAT) AS new_star_ratings
FROM red_route;


***After done cleaning, I transformed every columns into new table called as 'newbus_route'

In [ ]:
##Select everything from newbus_route
SELECT * FROM newbus_route;
